<a href="https://colab.research.google.com/github/sameepshrestha/Segmentation-of-glomeruli-using-Double-Unet/blob/main/DenseNet161freezeconcat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install --upgrade pip

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt 
import tifffile 
import os
import cv2 
import tensorflow as tf 
from tqdm import tqdm
import albumentations as A
import zipfile
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.layers import Input
from keras.layers import Activation
from keras.layers import BatchNormalization
from keras.layers import Dense
#from keras.layers import Flatten
from keras.layers import AveragePooling2D
from keras.layers.convolutional import Conv2D, MaxPooling2D, UpSampling2D, AveragePooling2D, Conv2DTranspose
from keras.layers.merge import concatenate #Concatenate (capital C) not working 
#from keras.utils.vis_utils import plot_model
from keras.layers import Dropout

from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.callbacks import CSVLogger
from tensorflow.keras.metrics import Recall, Precision
from keras.optimizers import Adam
%matplotlib inline 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def Augmentation(images,mask):
    def _transform(image,mask):
        transform=A.Compose([
         A.HorizontalFlip(p=0.25),
         A.VerticalFlip(p=0.25),
         A.RandomRotate90(p=.5),
         A.RandomContrast(limit=0.2, p=0.1),
        # A.OneOf([
         #   A.OpticalDistortion(p=0.3),
          #  A.GridDistortion(p=.1),
           # A.IAAPiecewiseAffine(p=0.3),
        #], p=0.2),
         #A.OneOf([
            #A.IAAAdditiveGaussianNoise(),
          #  A.GaussNoise(),
        #], p=0.2),
         # A.RandomScale(p=0.5),
         #A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, p=0.5),
         #A.RandomBrightnessContrast(p=0.5),
         A.Transpose(p=0.5),
         A.ElasticTransform(p=.4, alpha=120, sigma=120 * 0.05, alpha_affine=120 * 0.03),
    ])
        aug=transform(image=image,mask=mask)
        img=aug['image']
        img=tf.cast(img,tf.float64)
        msk=aug['mask']
        msk=tf.cast(msk,tf.float64)
        return img,msk
    image,mask=tf.numpy_function(_transform,[images,mask],[tf.float64,tf.float64])
    image.set_shape([IMAGE_SIZE, IMAGE_SIZE, 3])
    mask.set_shape([IMAGE_SIZE, IMAGE_SIZE, 2])
    
    return image,mask
from glob import glob
IMAGE_SIZE=512
AUTO = tf.data.experimental.AUTOTUNE
def load_data(path, split=0.2):
    images = sorted(glob(os.path.join(path, "train/*")))
    print(len(images))
    masks = sorted(glob(os.path.join(path, "masks/*")))
    print(len(masks))
    total_size = len(images)
    valid_size = int(split * total_size)

    train_x, valid_x = train_test_split(images, test_size=valid_size, random_state=42)
    train_y, valid_y = train_test_split(masks, test_size=valid_size, random_state=42)

    return (train_x, train_y), (valid_x, valid_y)
(train_x,train_y),(valid_x,valid_y)=load_data('/content/drive/MyDrive/HUBMAP')
train_size=len(train_x)
valid_size=len(valid_x)
def read_image(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = cv2.resize(x,(512,512),interpolation=cv2.INTER_AREA)
    x = x/255.0
    x=tf.cast(x,dtype=tf.float64)
    return x
def read_mask(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    x = cv2.resize(x,(512,512),interpolation=cv2.INTER_NEAREST)
    x = np.expand_dims(x, axis=-1)
    x = np.concatenate([x,x],axis=-1)
    x=tf.cast(x,dtype=tf.float64)
    return x
def parser(x,y):
    def _parse(x,y):
        x=read_image(x)
        y=read_mask(y)
        return x,y
    x,y = tf.numpy_function(_parse, [x,y], [tf.float64,tf.float64])
    x.set_shape([IMAGE_SIZE, IMAGE_SIZE, 3])
    y.set_shape([IMAGE_SIZE, IMAGE_SIZE, 2])
    return x,y
    
def tf_dataset(x, y, batch):
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.map(parser)
    dataset = dataset.map(Augmentation)
    dataset = dataset.repeat()
    dataset = dataset.shuffle(128,seed=0)
    dataset = dataset.batch(batch)

    return dataset
train_dataset=tf_dataset(train_x,train_y,batch=6)
valid_dataset=tf_dataset(valid_x,valid_y,batch=6)

3849
3849


In [ ]:
print(train_dataset)
print(valid_dataset)

<BatchDataset shapes: ((None, 512, 512, 3), (None, 512, 512, 2)), types: (tf.float64, tf.float64)>
<BatchDataset shapes: ((None, 512, 512, 3), (None, 512, 512, 2)), types: (tf.float64, tf.float64)>


In [ ]:
IMAGE_WIDTH=1024
IMAGE_HEIGHT=1024
IMAGE_CHANNELS=3

# loss functions 


In [ ]:
from tensorflow.keras.losses import binary_crossentropy
smooth = 1
def dice_coef(y_true, y_pred):
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

def binary_loss(y_true, y_pred):
    return binary_crossentropy(y_true, y_pred)

def binary_dice_loss(y_true, y_pred):
    d_loss= dice_loss(y_true, y_pred)
    b_loss= binary_loss(y_true, y_pred)
    loss=(d_loss+b_loss)
    return loss

def Tversky_Index(y_true,y_pred,smooth):
    y_true= tf.keras.layers.Flatten()(y_true)
    y_pred= tf.keras.layers.Flatten()(y_pred)
    true_positive= tf.reduce_sum(y_true*y_pred)
    false_negative=tf.reduce_sum(y_true*(1-y_pred))
    false_positive=tf.reduce_sum((1-y_true)*y_pred)
    alpha=.7
    TI=(true_positive+smooth)/(true_positive+alpha*false_negative+(1-alpha)*false_positive+smooth)
    return TI

def Tversky_loss(y_true,y_pred):
    return 1-Tversky_Index(y_true,y_pred,smooth=1)
def focal_Tversky_loss(y_true,y_pred):
    pt_1 = Tversky_Index(y_true, y_pred,smooth=1)
    gamma = 0.75
    return tf.math.pow((1-pt_1), gamma)




#model

In [ ]:

from keras.applications import VGG19
from keras.layers import GlobalAveragePooling2D 
from keras.layers import multiply, Reshape
from tensorflow.keras.optimizers import Nadam ,Adam
from tensorflow.keras.models import Model
from tensorflow.keras.applications import *

In [ ]:
def squeeze_excite_block(inputs, ratio=8):
    init = inputs
    channel_axis = -1
    filters = init.shape[channel_axis]
    se_shape = (1, 1, filters)

    se = GlobalAveragePooling2D()(init)
    se = Reshape(se_shape)(se)
    se = Dense(filters // ratio, activation='relu', kernel_initializer='he_normal', use_bias=False)(se)
    se = Dense(filters, activation='sigmoid', kernel_initializer='he_normal', use_bias=False)(se)

    x = multiply([init, se])
    return x

def conv_block(inputs, filters):
    x = inputs

    x = Conv2D(filters, (3, 3), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(filters, (3, 3), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = squeeze_excite_block(x)

    return x

def encoder1(inputs):
    skip_connections = [inputs]

    model = tf.keras.applications.DenseNet169(include_top=False, weights='imagenet',input_tensor=inputs)
    for layers in model.layers:
        layers.trainable=False
        
    #model.summary()
    names = ["conv1/relu", "pool2_conv", "pool3_conv", "pool4_conv"]
    for name in names:
        skip_connections.append(model.get_layer(name).output)

    output = model.get_layer("relu").output
    return output, skip_connections

def decoder1(inputs, skip_connections):
    num_filters = [ 256, 128, 64,64,32]
    skip_connections.reverse()
    x = inputs

    for i, f in enumerate(num_filters):
        x = UpSampling2D((2, 2), interpolation='bilinear')(x)
        x = concatenate([x, skip_connections[i]])
        x = conv_block(x, f)

    return x
def encoder2(inputs):
    num_filters = [32,64,64, 128, 256]
    skip_connections = []
    x = inputs

    for i, f in enumerate(num_filters):
        x = conv_block(x, f)
        skip_connections.append(x)
        x = MaxPooling2D((2, 2))(x)

    return x, skip_connections

def decoder2(inputs, skip_1, skip_2):
    num_filters = [256, 128, 64, 64,32]
    skip_2.reverse()
    x = inputs

    for i, f in enumerate(num_filters):
        x = UpSampling2D((2, 2), interpolation='bilinear')(x)
        x = concatenate([x, skip_1[i], skip_2[i]])
        x = conv_block(x, f)

    return x

def output_block(inputs):
    x = Conv2D(1, (1, 1), padding="same")(inputs)
    x = Activation('sigmoid')(x)
    return x

def Upsample(tensor, size):
    """Bilinear upsampling"""
    def _upsample(x, size):
        return tf.image.resize(images=x, size=size)
    return Lambda(lambda x: _upsample(x, size), output_shape=size)(tensor)
def ASPP(x, filter):
    shape = x.shape

    y1 = AveragePooling2D(pool_size=(shape[1], shape[2]))(x)
    y1 = Conv2D(filter, 1, padding="same")(y1)
    y1 = BatchNormalization()(y1)
    y1 = Activation("relu")(y1)
    y1 = UpSampling2D((shape[1], shape[2]), interpolation='bilinear')(y1)

    y2 = Conv2D(filter, 1, dilation_rate=1, padding="same", use_bias=False)(x)
    y2 = BatchNormalization()(y2)
    y2 = Activation("relu")(y2)

    y3 = Conv2D(filter, 3, dilation_rate=6, padding="same", use_bias=False)(x)
    y3 = BatchNormalization()(y3)
    y3 = Activation("relu")(y3)

    y4 = Conv2D(filter, 3, dilation_rate=12, padding="same", use_bias=False)(x)
    y4 = BatchNormalization()(y4)
    y4 = Activation("relu")(y4)

    y5 = Conv2D(filter, 3, dilation_rate=18, padding="same", use_bias=False)(x)
    y5 = BatchNormalization()(y5)
    y5 = Activation("relu")(y5)

    y = concatenate([y1, y2, y3, y4, y5])

    y = Conv2D(filter, 1, dilation_rate=1, padding="same", use_bias=False)(y)
    y = BatchNormalization()(y)
    y = Activation("relu")(y)

    return y
def build_model(shape):
    inputs = Input(shape)
    x, skip_1 = encoder1(inputs)
    x = ASPP(x, 64)
    x = decoder1(x, skip_1)
    outputs1 = output_block(x)

    x = inputs * outputs1

    x, skip_2 = encoder2(x)
    x = ASPP(x, 64)
    x = decoder2(x, skip_1, skip_2)
    outputs2 = output_block(x)
    outputs = concatenate([outputs1, outputs2])
    #output=Conv2D(1,(2,2),activation="sigmoid",padding="same")(outputs)
    model = Model(inputs, outputs)
    return model
model=build_model((512,512,3))
print(model.summary())

51879936/51877672 [==============================] - 2s 0us/step
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 512, 512, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 518, 518, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 256, 256, 64) 9408        zero_padding2d[0][0]             
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 256, 256, 64) 256         conv1/conv[0][0]                 
_____________________________

In [ ]:
# def lrfn(epoch):
#     if epoch <= lr_epoch:
#         lr = (lrmax-((lrmax-lrmin)/lr_epoch)*epoch)
#     else:
#         lr = lrmin
#     return lr
# lr_epoch=12
# lrmax=.0001
# lrmin=0.00001
# lr_callback = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose = True)
# Epochs=50
# rng = [i for i in range(12 if Epochs<12 else Epochs)]
# y = [lrfn(x) for x in rng]
# plt.plot(rng, y)
# print("Learning rate schedule: {:.3g} to {:.3g} to {:.3g}".format(y[0], max(y), y[-1]))

In [ ]:
import tensorflow as tf
model=tf.keras.models.load_model('/content/drive/MyDrive/HUBMAP/modelDEsnetFreeze.hd5',custom_objects={
        'dice_loss': dice_loss ,'dice_coef': dice_coef})


In [ ]:
filepath= '/content/drive/MyDrive/HUBMAP/modelDEsnetFreeze.hd5'
callbacks = [
    EarlyStopping(monitor='val_loss', patience=10),
    ModelCheckpoint(filepath,verbose=1,monitor='val_dice_coef',mode='max',save_best_only=True),
    CSVLogger("/content/drive/MyDrive/HUBMAP/dataDensenet161freeze.csv")
]

In [ ]:


model.compile(loss=dice_loss,optimizer=Adam(lr=.0000025),metrics=[dice_coef,Recall(),Precision()])

In [ ]:
t_steps=train_size//6
v_steps=valid_size//6
history=model.fit(train_dataset, 
                        epochs =20,
                        steps_per_epoch = t_steps,
                        validation_data = valid_dataset,
                        validation_steps = v_steps,
                        callbacks=callbacks
         )

Epoch 1/20
513/513 [==============================] - 3642s 7s/step - loss: 0.1803 - dice_coef: 0.8197 - recall: 0.8958 - precision: 0.8961 - val_loss: 0.1866 - val_dice_coef: 0.8134 - val_recall: 0.8847 - val_precision: 0.9017

Epoch 00001: val_dice_coef improved from -inf to 0.81339, saving model to /content/drive/MyDrive/HUBMAP/modelDEsnetFreeze.hd5
INFO:tensorflow:Assets written to: /content/drive/MyDrive/HUBMAP/modelDEsnetFreeze.hd5/assets
Epoch 2/20
513/513 [==============================] - 1144s 2s/step - loss: 0.1623 - dice_coef: 0.8377 - recall: 0.8977 - precision: 0.9033 - val_loss: 0.1649 - val_dice_coef: 0.8351 - val_recall: 0.8866 - val_precision: 0.8963

Epoch 00002: val_dice_coef improved from 0.81339 to 0.83513, saving model to /content/drive/MyDrive/HUBMAP/modelDEsnetFreeze.hd5
INFO:tensorflow:Assets written to: /content/drive/MyDrive/HUBMAP/modelDEsnetFreeze.hd5/assets
Epoch 3/20
513/513 [==============================] - 1141s 2s/step - loss: 0.1501 - dice_coef: 0.8

In [ ]:
plt.plot(history.history['val_dice_coef'],label='validation dice coefficient')
plt.plot(history.history['dice_coef'],label='dice coefficient')
plt.show()

In [ ]:
def rle_encode_less_memory(img):
    pixels = img.T.flatten()
    pixels[0] = 0
    pixels[-1] = 0
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 2
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)


def make_grid(shape, window=256, min_overlap=32):
    """
        Return Array of size (N,4), where N - number of tiles,
        2nd axis represente slices: x1,x2,y1,y2 
    """
    x, y = shape
    nx = x // (window - min_overlap) + 1
    x1 = np.linspace(0, x, num=nx, endpoint=False, dtype=np.int64)
    x1[-1] = x - window
    x2 = (x1 + window).clip(0, x)
    ny = y // (window - min_overlap) + 1
    y1 = np.linspace(0, y, num=ny, endpoint=False, dtype=np.int64)
    y1[-1] = y - window
    y2 = (y1 + window).clip(0, y)
    slices = np.zeros((nx,ny, 4), dtype=np.int64)
    
    for i in range(nx):
        for j in range(ny):
            slices[i,j] = x1[i], x2[i], y1[j], y2[j]    
    return slices.reshape(nx*ny,4)